# カイ二乗

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

skb = SelectKBest(chi2, k=20) # 20個とってくる。
skb.fit(X_train, y_train)
X_train_new = skb.transform(X_train)
# 選択した特徴をboolで表示。
skb.get_support()
data.feature_names[skb.get_support()]

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
clf = linear_model.LogisticRegression()

k_range = np.arange(1, 31)
scores = []
std = []

for k in k_range:

    ss = StratifiedKFold(n_splits=10, 
                         shuffle=True, 
                         random_state=2)
    score = []
    for train_index, val_index in ss.split(X_train,
                                           y_train):

        X_train2, X_val = X[train_index], X[val_index]
        y_train2, y_val = y[train_index], y[val_index]

        skb = SelectKBest(chi2, k=k)
        skb.fit(X_train2, y_train2)
        
        X_new_train2 = skb.transform(X_train2)
        X_new_val    = skb.transform(X_val) # testもtrainのskbで変換する。
        
        clf.fit(X_new_train2, y_train2)
        score.append( clf.score(X_new_val, y_val) )

    scores.append( np.array(score).mean() )
    std.append( np.array(score).std() )
    
scores = np.array(scores)
std = np.array(std)

In [ ]:
plt.plot(k_range, scores)
plt.errorbar(k_range, scores, yerr=std)
plt.ylabel("accuracy")

In [ ]:
best_k = k_range[np.argmax(scores)]
skb = SelectKBest(chi2, k=best_k)
skb.fit(X_train, y_train)
X_train_best = skb.transform(X_train)
X_test_best  = skb.transform(X_test)
clf.fit(X_train_best, y_train)
clf.score(X_test_best, y_test)